# English BERT

In [1]:
import numpy as np
import random as rn
import pandas as pd
import torch
from transformers import RobertaModel, TFRobertaForSequenceClassification, RobertaTokenizer
from torch.optim import AdamW, SGD
from torch import nn
from keras_preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import Adam
from torch.nn.utils.clip_grad import clip_grad_norm_
from torch.cuda.amp import GradScaler, autocast
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import itertools
import gc
import tqdm
from tqdm import tqdm
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score
from sklearn import metrics
import tensorflow as tf
assert tf.__version__ >= "2.0"


2023-12-16 13:56:54.530613: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-16 13:56:54.530679: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-16 13:56:54.532227: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-16 13:56:54.541600: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-16 13:56:55.656289: W tensorflow/compiler/tf2

In [23]:
from sklearn.base import BaseEstimator, TransformerMixin

model_name = 'roberta-base'
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = TFRobertaForSequenceClassification.from_pretrained(model_name, num_labels=2)

train_data = pd.read_csv("data/english/aclImdb/train.tsv", delimiter='\t')
test_data = pd.read_csv("data/english/aclImdb/test.tsv", delimiter='\t')

train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_data['text_a'], train_data['label'], test_size=0.2, random_state=1349
)

train_reviews = train_texts.tolist()
train_labels = train_labels.tolist()

val_reviews = val_texts.tolist()
val_labels = val_labels.tolist()

test_reviews = test_data['text_a'].tolist()
test_labels = test_data['label'].tolist()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

25000
25000


In [3]:
class RobertaPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, tokenizer, max_seq_length):
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length

    def transform(self, X, y):
        X_encoded = self.encode_texts(X)
        y_array = np.array(y)

        input_ids = tf.convert_to_tensor(X_encoded['input_ids'], dtype=tf.int32)
        attention_mask = tf.convert_to_tensor(X_encoded['attention_mask'], dtype=tf.int32)
        y_tensor = tf.convert_to_tensor(y_array, dtype=tf.int64)

        return {"input_ids": input_ids, "attention_mask": attention_mask}, y_tensor

    def encode_texts(self, texts):
        return self.tokenizer(texts, padding='max_length', truncation=True, max_length=self.max_seq_length, return_tensors="tf")

In [27]:
print(len(test_reviews))
print(len(test_labels))

25000
25000


In [26]:
MAX_SEQ_LEN = 512
preprocessor = RobertaPreprocessor(tokenizer, MAX_SEQ_LEN)
encoded_train, y_train = preprocessor.transform(train_reviews, train_labels)
encoded_val, y_val = preprocessor.transform(val_reviews, val_labels)
encoded_test, y_test = preprocessor.transform(test_reviews, test_labels)

opt = tf.keras.optimizers.Adam(learning_rate=3e-6, epsilon=1e-08)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)    
model.compile(optimizer=opt, loss=loss_fn, metrics=['accuracy'])

2
25000


In [28]:
print(len(encoded_test))
print(len(y_test))

2
25000


In [5]:
early_stopper = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=2, verbose=1, restore_best_weights=True
)

In [6]:
model.fit(
    encoded_train, y_train, epochs=3, batch_size=16, 
    validation_data=(encoded_val, y_val), 
    callbacks=[early_stopper], verbose=1
)

Epoch 1/3


2023-12-16 12:41:34.040718: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fecb28babf0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-16 12:41:34.040769: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100-SXM2-16GB, Compute Capability 7.0
2023-12-16 12:41:34.048879: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-16 12:41:34.070566: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8903
I0000 00:00:1702730494.185545   20908 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1250/1250 [==============================] - 829s 630ms/step - loss: 0.2493 - accuracy: 0.8871 - val_loss: 0.1516 - val_accuracy: 0.9466
Epoch 2/3
1250/1250 [==============================] - 779s 623ms/step - loss: 0.1442 - accuracy: 0.9498 - val_loss: 0.1453 - val_accuracy: 0.9490
Epoch 3/3
1250/1250 [==============================] - 779s 623ms/step - loss: 0.1116 - accuracy: 0.9617 - val_loss: 0.1441 - val_accuracy: 0.9506


In [7]:
predictions = model.predict(encoded_test)

782/782 [==============================] - 297s 375ms/step


In [29]:
logits = predictions.logits
print(len(y_test))
y_pred = np.argmax(logits, axis=1)
print(len(y_pred))
test_accuracy = metrics.accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {test_accuracy}")
print(metrics.classification_report(y_test, y_pred))

25000
25000
Test Accuracy: 0.95336
              precision    recall  f1-score   support

           0       0.97      0.94      0.95     12500
           1       0.94      0.97      0.95     12500

    accuracy                           0.95     25000
   macro avg       0.95      0.95      0.95     25000
weighted avg       0.95      0.95      0.95     25000


In [34]:
model_save_path = 'new_roberta.h5'
model.save_weights(model_save_path)
print(f"Model weights saved to {model_save_path}")


NameError: name 'model' is not defined

# French BERT

In [32]:
import numpy as np
import pandas as pd
from transformers import CamembertTokenizer, TFCamembertForSequenceClassification
from sklearn.metrics import classification_report
from sklearn import metrics
import tensorflow as tf
assert tf.__version__ >= "2.0"

In [7]:
model_name = "camembert-base"
tokenizer = CamembertTokenizer.from_pretrained(model_name)
model = TFCamembertForSequenceClassification.from_pretrained("camembert-base")

train_data = pd.read_csv("data/french/data/train.tsv", delimiter='\t')
val_data = pd.read_csv("data/french/data/val.tsv", delimiter='\t')
test_data = pd.read_csv("data/french/data/test.tsv", delimiter='\t')

train_reviews = train_data['text_a'].tolist()
train_labels = train_data['label'].tolist()

val_reviews = val_data['text_a'].tolist()
val_labels = val_data['label'].tolist()

test_reviews = test_data['text_a'].tolist()
test_labels = test_data['label'].tolist()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
All PyTorch model weights were used when initializing TFCamembertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFCamembertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
from sklearn.base import BaseEstimator, TransformerMixin

class CamembertPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, tokenizer, max_seq_length):
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length

    def transform(self, X, y):
        X_encoded = self.encode_reviews(self.tokenizer, X, self.max_seq_length)
        y_array = np.array(y)
        return X_encoded, y_array 

    def encode_reviews(self, tokenizer, reviews, max_length):
        token_ids = np.zeros(shape=(len(reviews), max_length), dtype=np.int32)
        for i, review in enumerate(reviews):
            encoded = tokenizer.encode(review, max_length=max_length)
            token_ids[i, 0:len(encoded)] = encoded
        attention_mask = (token_ids != 0).astype(np.int32)
        return {"input_ids": token_ids, "attention_mask": attention_mask}

In [16]:
opt = tf.keras.optimizers.Adam(learning_rate=5e-6, epsilon=1e-08)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)    
model.compile(optimizer=opt, loss=loss_fn, metrics=['accuracy'])

In [17]:
MAX_SEQ_LEN = 400
preprocessor = CamembertPreprocessor(tokenizer, MAX_SEQ_LEN)
encoded_train, y_train = preprocessor.transform(train_reviews, train_labels)
encoded_valid, y_val = preprocessor.transform(val_reviews, val_labels)
encoded_test, y_test = preprocessor.transform(test_reviews, test_labels)

early_stopper = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=2, verbose=1, restore_best_weights=True
)
model.fit(
    encoded_train, y_train, epochs=1, batch_size=4, 
    validation_data=(encoded_valid, y_val), 
    callbacks=[early_stopper], verbose=1
)

2023-12-16 09:00:06.970215: I external/local_xla/xla/service/service.cc:168] XLA service 0x55e8aac23b40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-16 09:00:06.970258: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100-SXM2-16GB, Compute Capability 7.0
2023-12-16 09:00:06.996611: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-16 09:00:07.053555: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8903
I0000 00:00:1702717207.177242   19244 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


39991/39991 [==============================] - 6023s 149ms/step - loss: 0.1080 - accuracy: 0.9606 - val_loss: 0.0791 - val_accuracy: 0.9714


In [26]:
predictions = model.predict(encoded_test)

625/625 [==============================] - 182s 291ms/step


In [33]:
logits = predictions.logits
print(logits.shape)
y_pred = np.argmax(logits, axis=1)
test_accuracy = metrics.accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {test_accuracy}")

(19992, 2)
Test Accuracy: 0.9718387354941976


In [35]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.98      0.97     10403
           1       0.98      0.96      0.97      9589

    accuracy                           0.97     19992
   macro avg       0.97      0.97      0.97     19992
weighted avg       0.97      0.97      0.97     19992


In [ ]:
model.save_weights(new_camembert.h5)

# Chinese BERT

In [2]:
import numpy as np
import pandas as pd
from transformers import BertTokenizer, TFBertForSequenceClassification

In [3]:
from sklearn.base import BaseEstimator, TransformerMixin

model_name = 'bert-base-chinese'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=2)

train_data = pd.read_csv("data/chinese/chnsenticorp-main/train.tsv", delimiter='\t')
val_data = pd.read_csv("data/chinese/chnsenticorp-main/dev.tsv", delimiter='\t')
test_data = pd.read_csv("data/chinese/chnsenticorp-main/test.tsv", delimiter='\t')

train_texts = train_data['text_a'].tolist()
train_labels = train_data['label'].tolist()
val_texts = val_data['text_a'].tolist()
val_labels = val_data['label'].tolist()
test_texts = test_data['text_a'].tolist()
test_labels = test_data['label'].tolist()

2023-12-16 13:57:04.221596: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-16 13:57:04.223000: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-16 13:57:04.223186: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [4]:
class BertPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, tokenizer, max_seq_length):
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length

    def transform(self, X, y):
        X_encoded = self.encode_texts(X)
        y_array = np.array(y)

        input_ids = tf.convert_to_tensor(X_encoded['input_ids'], dtype=tf.int32)
        attention_mask = tf.convert_to_tensor(X_encoded['attention_mask'], dtype=tf.int32)
        token_type_ids = tf.convert_to_tensor(X_encoded['token_type_ids'], dtype=tf.int32)  
        y_tensor = tf.convert_to_tensor(y_array, dtype=tf.int64)

        return {"input_ids": input_ids, "attention_mask": attention_mask, "token_type_ids": token_type_ids}, y_tensor

    def encode_texts(self, texts):
        return self.tokenizer(texts, padding='max_length', truncation=True, max_length=self.max_seq_length, return_tensors="tf")

In [5]:
MAX_SEQ_LEN = 512
preprocessor = BertPreprocessor(tokenizer, MAX_SEQ_LEN)
encoded_train, y_train = preprocessor.transform(train_texts, train_labels)
encoded_valid, y_val = preprocessor.transform(val_texts, val_labels)
encoded_test, y_test = preprocessor.transform(test_texts, test_labels)

opt = tf.keras.optimizers.Adam(learning_rate=5e-6, epsilon=1e-12)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)    
model.compile(optimizer=opt, loss=loss_fn, metrics=['accuracy'])

In [6]:
early_stopper = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, verbose=1, restore_best_weights=True)
model.fit(encoded_train, y_train, epochs=3, batch_size=16, validation_data=(encoded_valid, y_val), callbacks=[early_stopper], verbose=1)

Epoch 1/3


2023-12-16 13:57:50.193204: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f86182e27c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-16 13:57:50.193265: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100-SXM2-16GB, Compute Capability 7.0
2023-12-16 13:57:50.201969: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-16 13:57:50.223779: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8903
I0000 00:00:1702735070.344070   33095 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


572/572 [==============================] - 404s 622ms/step - loss: 0.2901 - accuracy: 0.8862 - val_loss: 0.1939 - val_accuracy: 0.9225
Epoch 2/3
572/572 [==============================] - 353s 617ms/step - loss: 0.1535 - accuracy: 0.9473 - val_loss: 0.1880 - val_accuracy: 0.9400
Epoch 3/3
572/572 [==============================] - 353s 617ms/step - loss: 0.1072 - accuracy: 0.9644 - val_loss: 0.1803 - val_accuracy: 0.9383


In [7]:
predictions = model.predict(encoded_test)

38/38 [==============================] - 17s 375ms/step


In [8]:
logits = predictions.logits
y_pred = np.argmax(logits, axis=1)
test_accuracy = metrics.accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {test_accuracy}")
print(metrics.classification_report(y_test, y_pred))

Test Accuracy: 0.9433333333333334
              precision    recall  f1-score   support

           0       0.96      0.92      0.94       598
           1       0.93      0.96      0.94       602

    accuracy                           0.94      1200
   macro avg       0.94      0.94      0.94      1200
weighted avg       0.94      0.94      0.94      1200


In [10]:
model_save_path = 'new_bert_cn.h5'
model.save_weights(model_save_path)
print(f"Model weights saved to {model_save_path}")

Model weights saved to new_bert_cn.h5
